# Libraries Importations

In [60]:
#We import the librarires

from selenium import webdriver
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time

# We define some basic and useful functions

In [42]:
def clickElement(category, classe, information):
    """This function allows you to click on an element of the HTML page"""
    element = driver.find_element_by_xpath(
        "//" + category + "[@" + classe + "='" + information + "']"
    )
    element.click()


def fillField(category, class_name, information, text):
    """This function allows you to fill in a field of the HTML page"""
    elem = driver.find_element_by_xpath(
        "//" + category + "[@" + class_name + "='" + information + "']"
    )
    elem.click()
    elem.clear()
    elem.send_keys(text)


def openLink(link):
    """This function allows you to open a link"""
    driver.get(link)
    time.sleep(6)
    

# This function will get all the href links that have the a field data-test = "job-link"
def getAll_JobDescriptionLinks():
    """This function allows you to get all the href links that have the a field data-test = "job-link" """
    # We get all the links of the page
    links = driver.find_elements_by_xpath("//a[@data-test='job-link']")
    # We get the href of all the links
    hrefs = [link.get_attribute("href") for link in links]
    return hrefs

def getAll_JobDescriptionLinks_Indeed():
    """This function allows you to get all the href links that have the a field data-test = "job-link" """
    # We get all the links of the page
    links = driver.find_elements_by_xpath("//a[@class='jcs-JobTitle css-jspxzf eu4oa1w0']")
    # We get the href of all the links
    hrefs = [link.get_attribute("href") for link in links]
    return hrefs


def assembleLinkGlassdoor(base, job_name, from_age):
    """This function allows you to assemble the link of the page to be scraped"""
    # Example: https://www.glassdoor.com/Job/machine-learning-jobs-SRCH_KO0,16.htm?fromAge=1
    job_name = job_name.replace(" ", "-")
    lenght_job_name = len(job_name)
    link = base + "Job/" + job_name + "-jobs-SRCH_KO0," + str(lenght_job_name) + ".htm?fromAge=" + str(from_age)
    return link


def assembleLinkIndeed(base, job_name, from_age, location):
    """This function allows you to assemble the link of the page to be scraped"""
    # Example: https://www.indeed.com/jobs?q=machine+learning+engineer&l=United+States&fromage=1
    job_name = job_name.replace(" ", "+")
    location = location.replace(" ", "+")
    link = base + "jobs?q=" + job_name + "&l=" + location + "&fromage=" + str(from_age)
    return link

# Glassdoor Webscrapping

In [35]:
# We initialize the driver through Firefox
global driver
driver = webdriver.Firefox()

# We set the link of the page we want to scrape
base = "https://www.glassdoor.com/"
job_name = "machine learning engineer"
from_age = 1

# We create the link
link = assembleLinkGlassdoor(base, job_name, from_age)
# We open the link
openLink(link)


# We get all the job offers for this search
job_offers = []
previous_page_offers = []
current_page_offers = []
while True:
    # We get all the job offers
    current_page_offers = getAll_JobDescriptionLinks()
    # We check if the current page offers are the same as the previous page offers
    if current_page_offers == previous_page_offers:
        break
    else:
        # We add the current page offers to the job offers
        job_offers += current_page_offers
        # We update the previous page offers
        previous_page_offers = current_page_offers
        # We click on the next button
        try:
            clickElement("span", "alt", "Close")
            time.sleep(1)
        except:
            pass
        clickElement("button", "aria-label", "Next")
        time.sleep(2)

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\617241832.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  links = driver.find_elements_by_xpath("//a[@data-test='job-link']")
C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\617241832.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_xpath(


In [36]:
len(job_offers)

32

In [38]:
job_offers[3]

'https://www.glassdoor.com/partner/jobListing.htm?pos=104&ao=1110586&s=58&guid=0000018865374708a00f3f1c838002e6&src=GD_JOB_AD&t=SR&vt=w&cs=1_306bb09c&cb=1685325301867&jobListingId=1008670927834&cpc=654405A9B1E0A9F5&jrtk=3-0-1h1ijehq6jii7801-1h1ijehqk284q000-a5890b1f0bf55a24--6NYlbfkN0C8DhssTksZ4tAWhh8LVIFF2qionQVVpONm6qYGpiaOiReQCnL4QXiir2PHzKHErG7_WxnSv0mVixlNr2xAMJM1dsiGdBv3zwL-QpFkYOMMJ6-B5xU5RKCYvaAXSaY3dAcjnOGT7RZ0Wsn9n043U-ud_DKBoFJFXAgnNzdo-MylnYJ1N3L3MVBRxJCQqkiDNvo_Njfmn1L_5ugiL7pduLZGuqKUroiJ3nXoz5KLnthY8BM4avmyIphzJlQjBxl78SvTKvVnmw9QOvyJUQOQo2BL9rhYRZs3JNYvMWDoUg1F0dvJSBYs_t98DLa396DgdN7iLMijYHpoVChWFAEfDe0ZTVN5oJBCDHKwXLz-9mlZMMXZbewTDbBwSs-FaHZMtgSELVyoA8oVDcq1tuAUsAl21PndLX9CmZ5Ih61727bgu9QKBaTGhvMzAompNlF9SSS5swIvnfohDRHVgS_pr2jcHJCIwiS7E92LoNY-PaDXEZ8OTwpL3ok-_NBSxp5zsh4bYRMYmRYuW4GxtWoq7ogXMegP6vkBe_49sapAWFOpaedA0yJZQt6C16PDL_JOdHvW8aZUOyarC51-D26ylrxqaGwpzU3CleLY37RuJyVqW717IA7MPelalQ_kyHVe9tjvsXE1M667M6hHrnA0_UkjcUCk2WIXeSYFfoOUaqGbepZPehtx3lm_rDB3WqLqBbOEEfOegCxrK

# Indeed WebScrapping

In [66]:
# We initialize the driver through Firefox
global driver
driver = uc.Chrome(use_subprocess=True)

# We set the link of the page we want to scrape
base = "https://www.indeed.com/"
job_name = "machine learning engineer"
from_age = 1
location = "United States"

# We create the link
link = assembleLinkIndeed(base, job_name, from_age, location)
# We open the link
openLink(link)
time.sleep(2)

# We get all the job offers for this search
job_offers = []

while True:
    # We get all the job offers of the first page
    current_page_offers = getAll_JobDescriptionLinks_Indeed()
    # We add the current page offers to the job offers
    job_offers += current_page_offers

    # We try to click on the next button
    try:
        clickElement("a", "data-testid", "pagination-page-next")
        time.sleep(2)
    except:
        break


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\4266679789.py:37: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  links = driver.find_elements_by_xpath("//a[@class='jcs-JobTitle css-jspxzf eu4oa1w0']")
C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\4266679789.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_xpath(
